In [2]:
import torch
from collections import OrderedDict

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams.update({'font.family': 'serif', 'font.size': 10})
matplotlib.rcParams['text.usetex'] = True
from matplotlib.lines import Line2D

from scipy.interpolate import griddata
from scipy import integrate
import time

np.random.seed(1234)

In this example, we solve a Lotka-Volterra Equation of the general form

\begin{align*}
\frac{dr}{dt} = \alpha r - \beta rp \\
\frac{dp}{dt} = \gamma rp - \delta p
\end{align*}

where $r$ is the number of prey, $p$ is the number of some predator, and $\alpha$, $\beta$, $\gamma$, and $\delta$ are real parameters describing the interactions of the two species. For the sake of simplicity, we assume that these parameters are all equal to one, i.e. we seek to solve the ODE pair

\begin{align*}
\frac{dr}{dt} = r - rp \\
\frac{dp}{dt} = rp - p
\end{align*}

In [3]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [4]:
# the deep neural network
class DNN(torch.nn.Module):
    
    def __init__(self, layers):
        
        super().__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
    
    
    def forward(self, x):
        
        # x = (t, y0)
        return self.layers(x)

In [5]:
# PINN: physics-informed neural network
class PINN():

    def __init__(self, X_pinn, X_semigroup, X_smooth, layers, T):

        # neural network architecture
        self.layers = layers
        self.dnn = DNN(layers).to(device)
        
        # semigroup PINN step time
        self.T = torch.tensor(T).float().to(device)

        # training data
        self.t_pinn = torch.tensor(X_pinn[:, :1], requires_grad=True).float().to(device)
        self.y_pinn = torch.tensor(X_pinn[:, 1:], requires_grad=True).float().to(device)
        
        self.s_semigroup = torch.tensor(X_semigroup[:, :1], requires_grad=True).float().to(device)
        self.t_semigroup = torch.tensor(X_semigroup[:, 1:2], requires_grad=True).float().to(device)
        self.y_semigroup = torch.tensor(X_semigroup[:, 2:], requires_grad=True).float().to(device)
        
        self.t_smooth = torch.tensor(X_smooth[:, :1], requires_grad=True).float().to(device)
        self.y_smooth = torch.tensor(X_smooth[:, 1:], requires_grad=True).float().to(device)
        
        # optimization
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), lr=1.0, max_iter=50000, max_eval=50000, 
            history_size=50, tolerance_grad=1e-5, tolerance_change=np.finfo(float).eps, 
            line_search_fn="strong_wolfe"
        )

        self.iter = 0
    
    
    def net_y(self, t, y0):
        
        # The M(t, y0) = y0 + t N(t, y0) scheme seems to drastically increase the accuracy
        # This works perfectly fine with automatic differentiation
        y = y0 + t * self.dnn(torch.cat([t, y0], dim=1))
        
        return y
    
    
    def net_derivative(self, t, y0):
        """
        Pytorch automatic differentiation to compute the derivative of the neural network
        """
        y = self.net_y(t, y0)
        
        # vectors for the autograd vector Jacobian product 
        # to compute the derivatives w.r.t. every output dimension
        vectors = [torch.zeros_like(y) for i in range(2)]
        
        for i, vec in enumerate(vectors):
            
            vec[:,i] = 1.
        
        # list of derivative tensors
        # the first entry is a tensor with \partial_t PINN(t, y0) for all (t, y0) in the batch,
        # each input (t, y0) corresponds to one row in each tensor
        derivatives = [
            torch.autograd.grad(
                y, t, 
                grad_outputs=vec,
                retain_graph=True,
                create_graph=True
            )[0]
            for vec in vectors
        ]
        
        return derivatives
    
    
    def loss_function(self):
        
        self.optimizer.zero_grad()
        
        y_pred = self.net_y(self.t_pinn, self.y_pinn)
        deriv_pred = self.net_derivative(self.t_pinn, self.y_pinn)
        
        """ Changed this """

        # in our case, dy1/dt = y1 - y1 * y2, dy2/dt = y1 * y2 -y2
        loss_pinn1 = torch.mean((deriv_pred[0] - y_pred[:,0:1] + y_pred[:,0:1] * y_pred[:,1:2]) ** 2)
        loss_pinn2 = torch.mean((deriv_pred[1] - y_pred[:,0:1] * y_pred[:,1:2] + y_pred[:,1:2]) ** 2)
        loss_pinn = loss_pinn1 + loss_pinn2 
        
        # The general semigroup loss for autonomous ODEs
        y_pred_tps = self.net_y(self.s_semigroup + self.t_semigroup, self.y_semigroup)
        y_pred_s = self.net_y(self.s_semigroup, self.y_semigroup)
        y_pred_restart = self.net_y(self.t_semigroup, y_pred_s)
        loss_semigroup = torch.mean((y_pred_tps - y_pred_restart) ** 2)
        
        # The smoothness loss
        y_pred_smooth = self.net_y(self.t_smooth, self.y_smooth)
        deriv_pred_below = self.net_derivative(self.t_smooth, self.y_smooth)
        deriv_pred_above = self.net_derivative(torch.zeros_like(self.t_smooth, requires_grad=True), y_pred_smooth)
        
        loss_smooth = .0
        
        for t1, t2 in zip(deriv_pred_below, deriv_pred_above):
            
            loss_smooth += torch.mean((t1 - t2) ** 2)
        
        loss = loss_pinn + loss_smooth + loss_semigroup
        
        loss.backward()
        self.iter += 1
        
        if self.iter % 100 == 0:
            print(
                f"Iter {self.iter}, Loss: {loss.item():.5f}, Loss_pinn: {loss_pinn.item():.5f} " \
                f"Loss_smooth: {loss_smooth.item():.5f}, Loss_semigroup: {loss_semigroup.item():.5f}"
            )
        
        return loss
    
    
    def train(self):
        
        self.dnn.train()
        self.optimizer.step(self.loss_function)
    
    
    def predict(self, t, y0):
        
        t = torch.tensor(t, requires_grad=True).float().to(device)
        y0 = torch.tensor(y0, requires_grad=True).float().to(device)
        
        self.dnn.eval()
        y = self.net_y(t, y0)
        y = y.detach().cpu().numpy()
        
        return y

### Setup data example

In [6]:
""" Changed the net to 6 x 64 """

layers = [3, 64, 64, 64, 64, 64, 64, 2]

T = 1
max_y0 = 5

# standard PINN loss function training samples
N_pinn = 10000
N_semigroup = 10000
N_smooth = 10000


t_pinn = np.random.uniform(0, T, (N_pinn, 1))
y_pinn = np.random.uniform(0, max_y0, (N_pinn, 2))
X_pinn = np.hstack([t_pinn, y_pinn])


r1 = np.random.uniform(0, 1, N_semigroup)
r2 = np.random.uniform(0, 1, N_semigroup)
s_semigroup, t_semigroup = np.sqrt(r1) * (1 - r2), r2 * np.sqrt(r1)
s_semigroup, t_semigroup = T * s_semigroup[:, np.newaxis], T * t_semigroup[:, np.newaxis]
y_semigroup = np.random.uniform(0, max_y0, (N_semigroup, 2))
X_semigroup = np.hstack([s_semigroup, t_semigroup, y_semigroup])


t_smooth = np.random.uniform(0, T, (N_smooth, 1))
y_smooth = np.random.uniform(0, max_y0, (N_smooth, 2))
X_smooth = np.hstack([t_smooth, y_smooth])

In [7]:
model = PINN(X_pinn, X_semigroup, X_smooth, layers, T)

In [8]:
%%time
               
model.train()

Iter 100, Loss: 2.08969, Loss_pinn: 1.55702 Loss_smooth: 0.52233, Loss_semigroup: 0.01035
Iter 200, Loss: 0.39984, Loss_pinn: 0.25900 Loss_smooth: 0.13734, Loss_semigroup: 0.00350
Iter 300, Loss: 0.14981, Loss_pinn: 0.09611 Loss_smooth: 0.05281, Loss_semigroup: 0.00089
Iter 400, Loss: 0.08359, Loss_pinn: 0.05276 Loss_smooth: 0.02986, Loss_semigroup: 0.00097
Iter 500, Loss: 0.04439, Loss_pinn: 0.02867 Loss_smooth: 0.01519, Loss_semigroup: 0.00054
Iter 600, Loss: 0.02971, Loss_pinn: 0.01980 Loss_smooth: 0.00962, Loss_semigroup: 0.00028
Iter 700, Loss: 0.02030, Loss_pinn: 0.01354 Loss_smooth: 0.00640, Loss_semigroup: 0.00037
Iter 800, Loss: 0.01506, Loss_pinn: 0.00975 Loss_smooth: 0.00508, Loss_semigroup: 0.00023
Iter 900, Loss: 0.01163, Loss_pinn: 0.00754 Loss_smooth: 0.00391, Loss_semigroup: 0.00018
Iter 1000, Loss: 0.00916, Loss_pinn: 0.00581 Loss_smooth: 0.00315, Loss_semigroup: 0.00020
Iter 1100, Loss: 0.00685, Loss_pinn: 0.00424 Loss_smooth: 0.00249, Loss_semigroup: 0.00012
Iter 120

In [9]:
# save the model

import os

path = os.getcwd()

torch.save(model.dnn.state_dict(), path + '/model_new.pt')

In [9]:
#model.dnn.load_state_dict(torch.load(path + '/model.pt'))

print(model.dnn.state_dict())

OrderedDict([('layers.layer_0.weight', tensor([[ 0.6039,  0.0127, -0.1380],
        [-0.2618,  0.1185,  0.0675],
        [-0.2961,  0.9878,  0.2246],
        [-0.1508,  0.3020,  0.5934],
        [-0.2849,  0.3702,  0.5397],
        [ 0.6226,  0.6795,  0.2412],
        [ 0.1156,  0.1567,  1.0192],
        [ 0.1775, -0.4479, -0.6348],
        [-0.3354, -0.3373, -0.2952],
        [-0.8970, -0.1149, -0.8433],
        [ 0.1445,  0.6219,  0.1104],
        [-0.2401, -0.2484, -0.6234],
        [ 0.0831, -1.0437, -0.0473],
        [ 0.2342,  0.2442, -0.1028],
        [ 0.4935,  0.7557,  0.1155],
        [-0.1002, -0.0389,  0.2192],
        [-0.2170,  0.0863,  0.1278],
        [-0.4783, -0.0154, -0.0582],
        [-0.1928, -0.0089,  0.3347],
        [-0.6260,  0.0084, -0.0961],
        [-0.3185,  0.7246, -0.0065],
        [ 0.4496, -0.5629, -0.1996],
        [ 0.1292,  0.1244,  0.5017],
        [-0.5663,  0.0378, -0.2409],
        [ 0.1310,  0.2961,  0.1681],
        [ 0.8657,  1.7634,  0.1343],

## Predict and Plot the Solution

In [10]:
def generate_figure(figsize, xlim, ylim):
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.spines[['top', 'right']].set_visible(False)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    return fig, ax


def plot_ode_solution(ax, y, *args, **kwargs):
    
    ax.plot(y[:,0], y[:,1], '.-', *args, **kwargs)
    
    return ax

In [11]:
def predict_standard(model, y0, max_t_pred, delta_t):
    
    times = np.linspace(0, max_t_pred, int(max_t_pred / delta_t) + 1)
    times = times[:,np.newaxis]
    
    y0 = np.array([y0 for _ in range(len(times))])
    trajectory =  model.predict(times, y0)
    
    return trajectory


def predict_dac(model, y0, max_t_pred, delta_t):
    """
    detla_t should devide model.max_t to guarantee equidistant steps
    """
    times = np.arange(0, model.T + delta_t, delta_t)[1:]
    times = times[:,np.newaxis]
    n_resets = int(np.ceil(max_t_pred / model.T))
    
    trajectory = np.array([y0])
    
    for _ in range(n_resets):
        
        y0 = trajectory[-1]
        y0 = np.array([y0 for _ in range(len(times))])
        segment =  model.predict(times, y0)
        trajectory = np.vstack([trajectory, segment])
    
    return trajectory

In [12]:
""" need to change the initial values here """

# Note that max_t in training is 1
y0 = [1., 0.15]
max_t_pred = 7.
delta_t = 0.05

validation_dac = predict_dac(model, y0, max_t_pred, delta_t)
validation_standard = predict_standard(model, y0, max_t_pred, delta_t)

In [16]:
""" true solution via solver (returns np.array) """


def func(t, r):
    x, y = r
    dx_t = x - x * y
    dy_t = x * y - y
    return dx_t, dy_t


def gen_truedata():
    t = np.linspace(0, max_t_pred, int(max_t_pred / delta_t) + 1)

    sol = integrate.solve_ivp(func, (0, 10), (y0[0], y0[1]), t_eval=t) 
    x_true, y_true = sol.y

    return np.stack((x_true, y_true), axis = 1)


true_solution = gen_truedata()

: 

: 

In [13]:
fig, ax = generate_figure(figsize=(8,8), xlim=[-7, 7], ylim=[-7, 7]) # probably need to change the limits

ax = plot_ode_solution(ax, validation_standard, label="Standard approach", color="#03468F")
ax = plot_ode_solution(ax, validation_dac, label="DaC approach", color="#A51C30")
ax = plot_ode_solution(ax, true_solution, label="true solution", color="orange")

plt.legend()
plt.savefig("proof_of_concept.pdf", bbox_inches="tight")
plt.show()

: 

: 